# Nissen et al., 2023: Severe 21st-century OA in Antarctic MPAs
#
# script to compute carbonate chemistry and save fields as netcdf files
# requires full model output
# note that some paths are hard-coded throughout the script
# note that there is the option to only compute carbonate chemistry for parts of the domain
# note that this script was run on Ollie (it was never updated to run ont he new machine)

In [ ]:
# some useful links:

# getting started with mocsy: http://ocmip5.ipsl.jussieu.fr/mocsy/fort.html
# getting started with using mocsy in python: http://ocmip5.ipsl.jussieu.fr/mocsy/pyth.html
# mocsy code: https://github.com/jamesorr/mocsy
# example notebooks: https://github.com/jamesorr/mocsy/tree/master/notebooks
# paper about mocsy: https://gmd.copernicus.org/articles/8/485/2015/gmd-8-485-2015.pdf 


In [1]:
#!jupyter nbconvert --to script save_carbonate_chemistry_as_netcdf.ipynb

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("python_gsw_py3/") # add GSW to search path -> python3 version
sys.path.append("mocsy/mocsy/")
#sys.path.append('/home/ollie/jhauck/py_fesom/modules/')
sys.path.append('/home/ollie/ncara/pyfesom2/')
import os
import pyfesom2 as pf
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from scipy.interpolate import griddata
from matplotlib import cm
from netCDF4 import Dataset, MFDataset
import pandas as pd
import mocsy
from gsw import p_from_z # get pressure from z and lat
import timeit
from numba import njit

In [ ]:
#-----
# get some general infor about mocsy
#-----

#print(mocsy.__version__)
#print(mocsy.__file__)
#dir(mocsy)
#print(mocsy.mvars.__doc__)
#print(mocsy.mbuffesm.__doc__)


In [ ]:
#-----
# mesh
#-----

mesh_id  = 'COARZE'  
meshpath = '/work/ollie/ncara/input/dgird_100/'

mesh = pf.load_mesh(meshpath,abg=[50, 15, -90],usepickle=False)

#depth = np.zeros(mesh.n2d) 
#print(mesh.y2)

file = meshpath+'/nod3d.out'
df=pd.read_csv(file, delim_whitespace=True, skiprows=1, \
            names=['node_number','x','y','z','flag'])
lat = df.y.values #[0:mesh.n2d] 
lon = df.x.values #[0:mesh.n2d]
depth_all = df.z.values #[0:mesh.n2d]
num_depths_all = len(np.unique(depth_all))
depths_avail = np.flipud(-1*np.unique(depth_all))

#------
# load cavity information
#------

file = meshpath+'/cavity_flag_nod2d.out'
df=pd.read_csv(file, delim_whitespace=True, skiprows=0, \
            names=['flag'])
cavities = df.flag.values

ind_cavities = np.where(cavities==1)[0]

# load topo 
print('Load topo')
path = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/Archive_Nissen2022_COMFORT1/'
ff = Dataset(path+'Nissen2022_FESOM_REcoM_mesh_information.nc')
topo = ff.variables['topo'][:]
ff.close()


In [ ]:
#-----
# load model output, calculate carbonate chemistry at each location, store in netcdf file
#-----
# what fields from model do I need?
#    tos, sos, alk (surface only), dic (surface only), sio2 (surface only), din (surface only), slp
#

#----
# some settings
#----

sim_list = ['simA-ssp585']#,'simCssp585','simB'] 
# 'simAssp585','simB','simCssp585','simD','simAssp245','simCssp245'

year_list = np.arange(1990,2100+1,1) #1980,2100+1,1) 
print(year_list)

# variables to save as netcdf 
# -> Note: if you want to save different variables, 
#          the code piece for writing these fields into a netcdf file needs to be adjusted
#vari_list = ['pH', 'co2', 'hco3', 'co3', 'gammadic', 'gammaalk', 'rf']
vari_list = ['pH', 'OmegaA','OmegaC','pco2','co3']

pressure_unit_conversion = 101325 # convert from Pa to atm

save_netcdf = True
# savepath: subdirectories will be created further down for each chosen simulation
savepath1    = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/carbonate_chemistry_3D_v2/'
if not os.path.exists(savepath1):
    print('Created '+savepath1)
    os.makedirs(savepath1)


#-----
# define the runIDs for all experiments of interest
#-----

def get_runID(year,simulation_id_long):
    # year is a number (not a string!)
    
    yy = year 
    
    if simulation_id_long in ['simA-ssp585']:
        if yy<1992: 
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_2'
        elif yy>2014: 
            runID='COARZE_AWICM_wREcoM_ssp585_incrCO2_1'
        else:
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_5'
    elif simulation_id_long in ['simA-ssp370']:
        if yy<1992: 
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_2'
        elif yy>2014: 
            runID='COARZE_AWICM_wREcoM_ssp370_incrCO2_1'
        else:
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_5'
    elif simulation_id_long in ['simA-ssp245']:
        if yy<1992: 
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_2'
        elif yy>2014: 
            runID='COARZE_AWICM_wREcoM_ssp245_incrCO2_1'
        else:
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_5'
    elif simulation_id_long in ['simA-ssp126']:
        if yy<1992: 
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_2'
        elif yy>2014: 
            runID='COARZE_AWICM_wREcoM_ssp126_incrCO2_1'
        else:
            runID='COARZE_AWICM_wREcoM_hist_incrCO2_5'
    elif simulation_id_long in ['simB']:
        if yy<2015: 
            runID='COARZE_AWICM_wREcoM_constClim_constCO2_1'
        else:
            runID='COARZE_AWICM_wREcoM_constClim_constCO2_2'
    return runID


In [ ]:
#----
# loop over simulations and years
#----

# option to reduce calculation to specific nodes:  
#indSO = np.where((mesh.y2<-38.0) & (cavities==0))[0]  # all nodes outside of cavity and south of 38°S
#print(indSO.shape)

indSO = np.where((mesh.y2<-60.0) & (topo<2000.0) & (cavities==0))[0]
print ('Number of nodes:',indSO.shape)

depths2000 = np.unique(-1*depth_all) #np.unique(mesh.zlev)
depths2000 = depths2000[depths2000<5500] # almost all depths in model; in the SO, no point is deeper than 5500m
num_depths = len(depths2000)
print('num_depths:',num_depths)

factor_conc = 1./1000. # to convert concentrations from mmol m-3 to mol m-3

save_netcdf = True

for ss in range(0,len(sim_list)):
    print('Process '+sim_list[ss])
    
    if save_netcdf: 
        sim_name = sim_list[ss]
        savepath    = savepath1+sim_name.replace('-', '')+'/'
        if not os.path.exists(savepath):
            print('Created '+savepath)
            os.makedirs(savepath)

    for yy in range(0,len(year_list)):
        print('year '+str(year_list[yy])+'...')
                
        #------
        # get runID for simulation & year
        #------
        runID = get_runID(year_list[yy],sim_list[ss])
        
        # paths to output
        path1 = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/'+runID+'/outdata/fesom/' 
        path2 = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/'+runID+'/outdata/recom/' 
        
        if save_netcdf: 
            fv = -9999
            for vv in range(0,len(vari_list)):
                netcdf_name = vari_list[vv]+'_fesom_'+str(year_list[yy])+'0101.nc'
                if not os.path.exists(savepath+netcdf_name):
                    print('Create file '+savepath+netcdf_name)
                    w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
                    w_nc_fid.mocsy_source = 'https://github.com/jamesorr/mocsy'
                    w_nc_fid.mocsy_dir    = '/home/ollie/ncara/scripts/mocsy/mocsy'
                    # create dimension & variable
                    w_nc_fid.createDimension('time', 12)  # monthly
                    w_nc_fid.createDimension('depth', num_depths)  # 3D
                    w_nc_fid.createDimension('nodes_2d', mesh.n2d) 
                    w_nc_var1 = w_nc_fid.createVariable(vari_list[vv], 'f4',('time','depth','nodes_2d'),fill_value=fv)
                    w_nc_var1.fesom_source = path1
                    w_nc_var1.recom_source = path2
                    #w_nc_var1.unit = 'degrees E'
                    w_nc_fid.close()
                    del netcdf_name
                    
        # load variables
        
        # open netcdf files
        f1 = Dataset(path1+'thetao_fesom_'+str(year_list[yy])+'0101.nc')
        f2 = Dataset(path1+'so_fesom_'+str(year_list[yy])+'0101.nc')
        f3 = Dataset(path2+'bgc03_fesom_'+str(year_list[yy])+'0101.nc') # alk
        f4 = Dataset(path2+'bgc02_fesom_'+str(year_list[yy])+'0101.nc') # dic
        f5 = Dataset(path2+'bgc18_fesom_'+str(year_list[yy])+'0101.nc') # sio2
        f6 = Dataset(path2+'bgc01_fesom_'+str(year_list[yy])+'0101.nc') # din
        f7 = Dataset(path1+'slp_fesom_'+str(year_list[yy])+'0101.nc')
        
        
        #start = timeit.timeit()
        
        
        for dd in range(0,num_depths):
            print('depth '+str(depths2000[dd])+'...')
            for mm in range(0,12):
                
                tos = f1.variables['thetao'][mm,dd,:]  # time x mesh.n2d
                sos = f2.variables['so'][mm,dd,:]
                alk = f3.variables['bgc03'][mm,dd,:] # surface only
                dic = f4.variables['bgc02'][mm,dd,:]
                sio = f5.variables['bgc18'][mm,dd,:]
                din = f6.variables['bgc01'][mm,dd,:]
                slp = f7.variables['slp'][mm,:]
                
                # mask nodes in cavity and outside of SO
                #indSO = np.where((mesh.y2<-60.0) & (topo<2000.0) & (cavities==0))[0]
                dic = np.ma.masked_where(cavities==1, dic)
                dic = np.ma.masked_where(mesh.y2>-59, dic)
                #dic = np.ma.masked_where(topo>00, dic)
                            
                # convert slp from Pa to atm: 
                slp = slp*pressure_unit_conversion

                # only do calculation for available nodes at current depth
                ind_av = dic.mask==False
                
                if depths_avail[dd]<0:
                    pressure = p_from_z(depths_avail[dd],mesh.y2[ind_av])
                elif depths_avail[dd]>=0:
                    pressure = p_from_z(-1*depths_avail[dd],mesh.y2[ind_av]) # to make sure that pressure is >0

                pH = np.nan*np.zeros(mesh.n2d)
                pco2 = np.nan*np.zeros(mesh.n2d)
                fco2 = np.nan*np.zeros(mesh.n2d)
                co2 = np.nan*np.zeros(mesh.n2d)
                hco3 = np.nan*np.zeros(mesh.n2d)
                co3 = np.nan*np.zeros(mesh.n2d)
                OmegaA = np.nan*np.zeros(mesh.n2d)
                OmegaC = np.nan*np.zeros(mesh.n2d)
                BetaD = np.nan*np.zeros(mesh.n2d)
                DENis = np.nan*np.zeros(mesh.n2d)
                p = np.nan*np.zeros(mesh.n2d)
                Tis = np.nan*np.zeros(mesh.n2d)

                gammadic = np.nan*np.zeros(mesh.n2d)
                betadic = np.nan*np.zeros(mesh.n2d)
                omegadic = np.nan*np.zeros(mesh.n2d)
                gammaalk = np.nan*np.zeros(mesh.n2d)
                betaalk = np.nan*np.zeros(mesh.n2d)
                omegaalk = np.nan*np.zeros(mesh.n2d)
                rf = np.nan*np.zeros(mesh.n2d)

                print('Min/Max DIC:',np.min(dic[ind_av]),np.max(dic[ind_av]))
                print('Min/Max sos:',np.min(sos[ind_av]),np.max(sos[ind_av]))

                pH[ind_av],pco2[ind_av],fco2[ind_av],co2[ind_av],\
                        hco3[ind_av],co3[ind_av],OmegaA[ind_av],OmegaC[ind_av],\
                        BetaD[ind_av],DENis[ind_av],p[ind_av],Tis[ind_av] = mocsy.mvars(tos[ind_av],
                                                sos[ind_av], alk[ind_av]*factor_conc, dic[ind_av]*factor_conc,\
                                                    sio[ind_av]*factor_conc, (din[ind_av]/16.0)*factor_conc, 
                                                 slp[ind_av], pressure, mesh.y2[ind_av],optcon='mol/m3', optt='Tpot', 
                                                 optp='db', optb="u74", optk1k2='l', optkf="dg", optgas="Pinsitu")

                #gammadic[ind_av],betadic[ind_av],omegadic[ind_av],\
                #        gammaalk[ind_av],betaalk[ind_av],omegaalk[ind_av],rf[ind_av] = mocsy.mbuffesm(tos[ind_av], sos[ind_av],
                #                                        alk[ind_av], dic[ind_av], sio[ind_av], din[ind_av]/16.0,
                #                                        slp[ind_av], pressure, mesh.y2[ind_av],
                #                                        optcon='mol/m3', optt='Tpot',optp='db',
                #                                        optb="u74", optk1k2='l', optkf="dg", optgas="Pinsitu")
                
                if (dd==0) | (dd==num_depths-1): # print some info for first and last depth level
                    print('Depth level '+str(dd+1))
                    print('Month '+str(mm+1))
                    print('Min/Max OmegaA:',np.nanmin(OmegaA[ind_av]),np.nanmax(OmegaA[ind_av]))
                    print('Min/Max OmegaC:',np.nanmin(OmegaC[ind_av]),np.nanmax(OmegaC[ind_av]))
                    print('Min/Max pH:',np.nanmin(pH[ind_av]),np.nanmax(pH[ind_av]))
                    print('Min/Max pco2:',np.nanmin(pco2[ind_av]),np.nanmax(pco2[ind_av]))
                    print('Min/Max co3:',np.nanmin(co3[ind_av]),np.nanmax(co3[ind_av]))

                del fco2,BetaD,DENis,p,Tis
                del co2, hco3, gammadic, gammaalk, rf
                del betadic,omegadic,betaalk,omegaalk,pressure

                # to store: pH, co2, hco3, co3, gammadic, gammaalk, rf
                if save_netcdf: 
                    for vv in range(0,len(vari_list)):
                        netcdf_name = vari_list[vv]+'_fesom_'+str(year_list[yy])+'0101.nc'
                        w_nc_fid = Dataset(savepath+netcdf_name, 'r+', format='NETCDF4_CLASSIC') 
                        if vari_list[vv] in ['pH']:
                            pH[np.isnan(pH)]=fv
                            w_nc_fid.variables[vari_list[vv]][mm,dd,:] = pH
                        elif vari_list[vv] in ['OmegaA']:
                            OmegaA[np.isnan(OmegaA)]=fv
                            w_nc_fid.variables[vari_list[vv]][mm,dd,:] = OmegaA
                        elif vari_list[vv] in ['OmegaC']:
                            OmegaC[np.isnan(OmegaC)]=fv
                            w_nc_fid.variables[vari_list[vv]][mm,dd,:] = OmegaC
                        elif vari_list[vv] in ['pco2']:
                            pco2[np.isnan(pco2)]=fv
                            w_nc_fid.variables[vari_list[vv]][mm,dd,:] = pco2
                        elif vari_list[vv] in ['co3']:
                            pco2[np.isnan(co3)]=fv
                            w_nc_fid.variables[vari_list[vv]][mm,dd,:] = co3
                        w_nc_fid.close()  
                        del netcdf_name
                    print(sim_list[ss]+': Successfully stored depth level '+str(dd+1)+' of month '+\
                          str(mm+1)+' of year '+str(year_list[yy]))

                del pH,OmegaA,OmegaC,pco2,co3
            del tos,sos,alk,dic,din,sio,slp
                
        # close netcdf files
        f1.close()
        f2.close()
        f3.close()
        f4.close()
        f5.close()
        f6.close()
        f7.close()
            
        #end = timeit.timeit()
        #print(end - start)
        
print('done')

In [ ]:

test_plot = False

if test_plot:
    path1 = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/carbonate_chemistry_3D/simAssp585/'
    path2 = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/carbonate_chemistry_3D_v2/simAssp585/'

    file1 = 'pH_fesom_19900101.nc'

    ff1 = Dataset(path1+file1)
    ff2 = Dataset(path2+file1)

    data1 = ff1.variables['pH'][0,0,:]
    data2 = ff2.variables['pH'][0,0,:]

    ff1.close()
    ff2.close()

    print(data1.shape)
    print(data2.shape)

    cs=1

    fig7= plt.figure(num=18, figsize=(10,4), dpi=180, facecolor='w', edgecolor='k')

    plt.scatter(mesh.x2,mesh.y2,cs,data1)

    plt.show()

    fig7= plt.figure(num=18, figsize=(10,4), dpi=180, facecolor='w', edgecolor='k')

    plt.scatter(mesh.x2,mesh.y2,cs,data2)

    plt.show()

